In [46]:
from transformers import AutoTokenizer, AutoModel,  DistilBertForQuestionAnswering
import torch

## Loading the model from the path

In [47]:
model_path = "model" #provide model path here (the model folder path)

In [48]:
model = AutoModel.from_pretrained(model_path)
model = DistilBertForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Some weights of the model checkpoint at model were not used when initializing DistilBertModel: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Providing context and question

In [49]:
context1 = 'We the team consisting of bhanu , keshav , courtney , morgan have built a model to answer question. We are studetns of northeastern university. NLP is our favourite subject'
question12 = 'which university does the team study in'
question11 = "who are the team members?"
context2 = "Charles Babbage, an English mechanical engineer and polymath, originated the concept of a programmable computer. Considered the father of the computer, he conceptualized and invented the first mechanical computer in the early 19th century. After working on his revolutionary difference engine, designed to aid in navigational calculations, in 1833 he realized that a much more general design, an Analytical Engine, was possible. The input of programs and data was to be provided to the machine via punched cards, a method being used at the time to direct mechanical looms such as the Jacquard loom. For output, the machine would have a printer, a curve plotter and a bell. The machine would also be able to punch numbers onto cards to be read in later. The Engine incorporated an arithmetic logic unit, control flow in the form of conditional branching and loops, and integrated memory, making it the first design for a general-purpose computer that could be described in modern terms as Turing-complete."
question2 = "when was the first computer invented" 
context3 = 'many important complexity classes can be defined by bounding the time or space used by the algorithm some important complexity classes of decision problems defined in this manner are the following'
question3 = 'what function is used by algorithms to define measurements like time or space'
context4 = 'the neighborhood of sunnyside is on fresnos far southeast side bounded by chestnut avenue to the west its major thoroughfares are kings canyon avenue and clovis avenue although parts of sunnyside are within the city of fresno much of the neighborhood is a county island within fresno county largely developed in the 1950s through the 1970s it has recently experienced a surge in new home construction it is also the home of the sunnyside country club which maintains a golf course designed by william p bell'
question4 = 'who designed the golf course located at the sunnyside country club'

### Running inference on the above question and answers

In [51]:
def inference(question,context):
    encoding = tokenizer.encode_plus(question, context)

    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

    output = model(torch.tensor([input_ids]), attention_mask=torch.tensor([attention_mask]))

    start_scores = output['start_logits']
    end_scores = output['end_logits']

    ans_tokens = input_ids[torch.argmax(start_scores) : torch.argmax(end_scores)+1]
    answer_tokens = tokenizer.convert_ids_to_tokens(ans_tokens , skip_special_tokens=True)

    print("\nContext:", context)
    print ("\nQuestion: ",question)
    print ("\nAnswer Tokens: ")
    print (answer_tokens)

    answer_tokens_to_string = tokenizer.convert_tokens_to_string(answer_tokens)

    print ("\nAnswer: ",answer_tokens_to_string)


In [52]:
inference(question3,context3) # example from the test dataset


Context: many important complexity classes can be defined by bounding the time or space used by the algorithm some important complexity classes of decision problems defined in this manner are the following

Question:  what function is used by algorithms to define measurements like time or space

Answer Tokens: 
['bound', '##ing', 'the', 'time', 'or', 'space']

Answer:  bounding the time or space


In [53]:
inference(question4,context4) # example from the test dataset


Context: the neighborhood of sunnyside is on fresnos far southeast side bounded by chestnut avenue to the west its major thoroughfares are kings canyon avenue and clovis avenue although parts of sunnyside are within the city of fresno much of the neighborhood is a county island within fresno county largely developed in the 1950s through the 1970s it has recently experienced a surge in new home construction it is also the home of the sunnyside country club which maintains a golf course designed by william p bell

Question:  who designed the golf course located at the sunnyside country club

Answer Tokens: 
['william', 'p', 'bell']

Answer:  william p bell


In [54]:
inference(question2,context2) # our own example context from wikipedia article, question our own


Context: Charles Babbage, an English mechanical engineer and polymath, originated the concept of a programmable computer. Considered the father of the computer, he conceptualized and invented the first mechanical computer in the early 19th century. After working on his revolutionary difference engine, designed to aid in navigational calculations, in 1833 he realized that a much more general design, an Analytical Engine, was possible. The input of programs and data was to be provided to the machine via punched cards, a method being used at the time to direct mechanical looms such as the Jacquard loom. For output, the machine would have a printer, a curve plotter and a bell. The machine would also be able to punch numbers onto cards to be read in later. The Engine incorporated an arithmetic logic unit, control flow in the form of conditional branching and loops, and integrated memory, making it the first design for a general-purpose computer that could be described in modern terms as Tu

In [55]:
inference(question11,context1) # our own context, our own question


Context: We the team consisting of bhanu , keshav , courtney , morgan have built a model to answer question. We are studetns of northeastern university. NLP is our favourite subject

Question:  who are the team members?

Answer Tokens: 
['b', '##han', '##u', ',', 'ke', '##sha', '##v', ',', 'courtney', ',', 'morgan']

Answer:  bhanu, keshav, courtney, morgan


In [56]:
inference(question12,context1) # our own context, our own question


Context: We the team consisting of bhanu , keshav , courtney , morgan have built a model to answer question. We are studetns of northeastern university. NLP is our favourite subject

Question:  which university does the team study in

Answer Tokens: 
['northeastern', 'university']

Answer:  northeastern university
